In [112]:
import requests
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import re

In [190]:
def fetch_endodontists_by_city_state(city, state):
    base_url = "https://npiregistry.cms.hhs.gov/api/"
    params = {
        "version": "2.1",
        "enumeration_type": "NPI-1",
        "taxonomy_description": "Endodontic",
        "city": city,
        "state": state,
    }

    response = requests.get(base_url, params=params)
    data = response.json()
    
    firstname = []
    lastname = []
    address_1 = []
    address_2 = []
    city_list = []
    state_list = []
    zip_code = []
    desc = []
    
    for i in range(len(data['results'])):
        firstname.append(data['results'][i]['basic']['first_name'])
        lastname.append(data['results'][i]['basic']['last_name'])
        address_1.append(data['results'][i]['addresses'][0]['address_1'].lower())
        try:
            address_2.append(data['results'][i]['addresses'][0]['address_2'].lower())
           
        except KeyError:
      
            address_2.append('')  # Use empty string if address_2 is missing
        city_list.append(data['results'][i]['addresses'][0]['city'].lower())
        state_list.append(data['results'][i]['addresses'][0]['state'].lower())
        zip_code.append(data['results'][i]['addresses'][0]['postal_code'][:5])
        desc.append(data['results'][i]['taxonomies'][0]['desc'].lower())

    df = pd.DataFrame({
        'firstname': firstname,
        'lastname': lastname,
        'address_1': address_1, 
        'address_2': address_2, 
        'city': city_list, 
        'state': state_list, 
        'zip_code': zip_code, 
        'desc': desc  
    }).astype({
    'firstname': 'string',
    'lastname': 'string',
    'address_1': 'string',
    'address_2': 'string',
    'city': 'string',
    'state': 'string',
    'zip_code': 'string',  # using string to preserve leading zeroes
    'desc': 'string'
    })

   

    df = df[(df['city']==city.lower()) & (df['state']==state.lower())]
    return df

# Example usage
df_endodontists = fetch_endodontists_by_city_state("greensboro", "nc")
df_endodontists

,firstname,lastname,address_1,address_2,city,state,zip_code,desc
0,ELIZABETH,APPLEBAUM,200 e wendover ave,suite b,greensboro,nc,27401,"dentist, endodontics"
1,MATTHEW,BAIRD,1602 benjamin pkwy,,greensboro,nc,27408,"dentist, endodontics"
2,CHARLES,BEAVERS,2600 oakcrest ave ste b,,greensboro,nc,27408,"dentist, endodontics"
3,RICHARD,BEAVERS,2600 oakcrest ave ste b,,greensboro,nc,27408,"dentist, endodontics"
4,NATHAN,HORWITZ,1602 benjamin pkwy ste a,,greensboro,nc,27408,"dentist, endodontics"
5,KEVIN,KEATING,2600 oakcrest ave ste b,,greensboro,nc,27408,"dentist, endodontics"
7,HAROLD,MOHORN,1602 benjamin pkwy,,greensboro,nc,27408,"dentist, endodontics"
8,STEVEN,MOHORN,1602 benjamin pkwy,,greensboro,nc,27408,"dentist, endodontics"
9,ZACKYRY,MOHORN,1602 benjamin pkwy ste a,,greensboro,nc,27408,"dentist, endodontics"


In [117]:
df_endodontists['clean_address_1'] = df_endodontists['address_1'].apply(
   lambda x: re.sub(r'\s*(ste|suite|apt|unit|bldg|fl)\s*\w+', '', str(x), flags=re.IGNORECASE))

df_endodontists['full_address'] = df_endodontists['clean_address_1']  + ', ' + df_endodontists['city'] + ', ' + df_endodontists['state'] + ', ' + df_endodontists['zip_code'] + ', ' + 'USA'

In [104]:
pd.set_option('display.max_colwidth', None)

In [118]:
df_endodontists

,credentials,firstname,lastname,address_1,address_2,city,state,zip_code,desc,clean_address_1,full_address
0,D.M.D,ELIZABETH,APPLEBAUM,200 e wendover ave,suite b,greensboro,nc,27401,"dentist, endodontics",200 e wendover ave,"200 e wendover ave, greensboro, nc, 27401, USA"
1,,MATTHEW,BAIRD,1602 benjamin pkwy,,greensboro,nc,27408,"dentist, endodontics",1602 benjamin pkwy,"1602 benjamin pkwy, greensboro, nc, 27408, USA"
2,,CHARLES,BEAVERS,2600 oakcrest ave ste b,,greensboro,nc,27408,"dentist, endodontics",2600 oakcrest ave,"2600 oakcrest ave, greensboro, nc, 27408, USA"
3,,RICHARD,BEAVERS,2600 oakcrest ave ste b,,greensboro,nc,27408,"dentist, endodontics",2600 oakcrest ave,"2600 oakcrest ave, greensboro, nc, 27408, USA"
4,,NATHAN,HORWITZ,1602 benjamin pkwy ste a,,greensboro,nc,27408,"dentist, endodontics",1602 benjamin pkwy,"1602 benjamin pkwy, greensboro, nc, 27408, USA"
5,,KEVIN,KEATING,2600 oakcrest ave ste b,,greensboro,nc,27408,"dentist, endodontics",2600 oakcrest ave,"2600 oakcrest ave, greensboro, nc, 27408, USA"
7,,HAROLD,MOHORN,1602 benjamin pkwy,,greensboro,nc,27408,"dentist, endodontics",1602 benjamin pkwy,"1602 benjamin pkwy, greensboro, nc, 27408, USA"
8,,STEVEN,MOHORN,1602 benjamin pkwy,,greensboro,nc,27408,"dentist, endodontics",1602 benjamin pkwy,"1602 benjamin pkwy, greensboro, nc, 27408, USA"
9,,ZACKYRY,MOHORN,1602 benjamin pkwy ste a,,greensboro,nc,27408,"dentist, endodontics",1602 benjamin pkwy,"1602 benjamin pkwy, greensboro, nc, 27408, USA"


In [119]:
geolocator = Nominatim(user_agent="streamlit-map-app")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [135]:
location = geolocator.geocode('2415 NE 134th St, Vancouver, WA, 98686, USA', timeout=10)
location

In [108]:
location

In [120]:
# Geocode addresses
df_endodontists['location'] = df_endodontists['full_address'].apply(geocode)
df_endodontists['lat'] = df_endodontists['location'].apply(lambda loc: loc.latitude if loc else None)
df_endodontists['lon'] = df_endodontists['location'].apply(lambda loc: loc.longitude if loc else None)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('2600 oakcrest ave, greensboro, nc, 27408, USA',), **{}).
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/urllib3/connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/urllib3/connection.py", line 464, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/http/client.py", line 1423, in getresponse
    response.begin()
  File "/opt/anaconda3/lib/python3.12/http/client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/http/client.py", line 292, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/op

In [121]:
df_endodontists

,credentials,firstname,lastname,address_1,address_2,city,state,zip_code,desc,clean_address_1,full_address,location,lat,lon
0,D.M.D,ELIZABETH,APPLEBAUM,200 e wendover ave,suite b,greensboro,nc,27401,"dentist, endodontics",200 e wendover ave,"200 e wendover ave, greensboro, nc, 27401, USA","(200, East Wendover Avenue, Proximity, Greensboro, Guilford County, North Carolina, 27401, United States, (36.088112, -79.7884145))",36.088112,-79.788415
1,,MATTHEW,BAIRD,1602 benjamin pkwy,,greensboro,nc,27408,"dentist, endodontics",1602 benjamin pkwy,"1602 benjamin pkwy, greensboro, nc, 27408, USA","(1602, Benjamin Parkway, Guilford Hills, Greensboro, Guilford County, North Carolina, 27408, United States, (36.1041837, -79.8305665))",36.104184,-79.830567
2,,CHARLES,BEAVERS,2600 oakcrest ave ste b,,greensboro,nc,27408,"dentist, endodontics",2600 oakcrest ave,"2600 oakcrest ave, greensboro, nc, 27408, USA","(2600A, Oakcrest Avenue, Guilford Hills, Greensboro, Guilford County, North Carolina, 27408, United States, (36.104521, -79.8286949))",36.104521,-79.828695
3,,RICHARD,BEAVERS,2600 oakcrest ave ste b,,greensboro,nc,27408,"dentist, endodontics",2600 oakcrest ave,"2600 oakcrest ave, greensboro, nc, 27408, USA","(2600A, Oakcrest Avenue, Guilford Hills, Greensboro, Guilford County, North Carolina, 27408, United States, (36.104521, -79.8286949))",36.104521,-79.828695
4,,NATHAN,HORWITZ,1602 benjamin pkwy ste a,,greensboro,nc,27408,"dentist, endodontics",1602 benjamin pkwy,"1602 benjamin pkwy, greensboro, nc, 27408, USA","(1602, Benjamin Parkway, Guilford Hills, Greensboro, Guilford County, North Carolina, 27408, United States, (36.1041837, -79.8305665))",36.104184,-79.830567
5,,KEVIN,KEATING,2600 oakcrest ave ste b,,greensboro,nc,27408,"dentist, endodontics",2600 oakcrest ave,"2600 oakcrest ave, greensboro, nc, 27408, USA","(2600A, Oakcrest Avenue, Guilford Hills, Greensboro, Guilford County, North Carolina, 27408, United States, (36.104521, -79.8286949))",36.104521,-79.828695
7,,HAROLD,MOHORN,1602 benjamin pkwy,,greensboro,nc,27408,"dentist, endodontics",1602 benjamin pkwy,"1602 benjamin pkwy, greensboro, nc, 27408, USA","(1602, Benjamin Parkway, Guilford Hills, Greensboro, Guilford County, North Carolina, 27408, United States, (36.1041837, -79.8305665))",36.104184,-79.830567
8,,STEVEN,MOHORN,1602 benjamin pkwy,,greensboro,nc,27408,"dentist, endodontics",1602 benjamin pkwy,"1602 benjamin pkwy, greensboro, nc, 27408, USA","(1602, Benjamin Parkway, Guilford Hills, Greensboro, Guilford County, North Carolina, 27408, United States, (36.1041837, -79.8305665))",36.104184,-79.830567
9,,ZACKYRY,MOHORN,1602 benjamin pkwy ste a,,greensboro,nc,27408,"dentist, endodontics",1602 benjamin pkwy,"1602 benjamin pkwy, greensboro, nc, 27408, USA","(1602, Benjamin Parkway, Guilford Hills, Greensboro, Guilford County, North Carolina, 27408, United States, (36.1041837, -79.8305665))",36.104184,-79.830567


In [160]:
base_url = "https://npiregistry.cms.hhs.gov/api/"
params = {
    "version": "2.1",
    "enumeration_type": "NPI-1",
    "taxonomy_description": "Endodontic",
    "city": 'los angeles',
    "state": 'CA',
}

response = requests.get(base_url, params=params)
data = response.json()

In [168]:
print(data['results'][0])

{'created_epoch': '1676286224000', 'enumeration_type': 'NPI-1', 'last_updated_epoch': '1676286224000', 'number': '1265139075', 'addresses': [{'country_code': 'US', 'country_name': 'United States', 'address_purpose': 'MAILING', 'address_type': 'DOM', 'address_1': '1031 W 34TH ST STE 401', 'city': 'LOS ANGELES', 'state': 'CA', 'postal_code': '900893603', 'telephone_number': '213-740-2012'}, {'country_code': 'US', 'country_name': 'United States', 'address_purpose': 'LOCATION', 'address_type': 'DOM', 'address_1': '1031 W 34TH ST STE 401', 'city': 'LOS ANGELES', 'state': 'CA', 'postal_code': '900893603', 'telephone_number': '213-740-2012'}], 'practiceLocations': [], 'basic': {'first_name': 'KHALID', 'last_name': 'ALHEZAIMI', 'credential': 'BDS', 'sole_proprietor': 'NO', 'sex': 'M', 'enumeration_date': '2023-02-13', 'last_updated': '2023-02-13', 'certification_date': '2023-02-12', 'status': 'A'}, 'taxonomies': [{'code': '1223E0200X', 'taxonomy_group': '', 'desc': 'Dentist, Endodontics', 'sta

In [162]:
len(data['results'])

10

In [171]:
import requests

url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {"location":"Greensboro, NC","status_type":"ForRent","home_type":"Apartments"}

headers = {
    "X-RapidAPI-Key": "YOUR_RAPIDAPI_KEY",
    "X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

data = response.json()

In [172]:
data

{'message': 'You are not subscribed to this API.'}

In [173]:
df = pd.read_excel('fy2025_safmrs_revised.xlsx')

In [194]:
df_clean = df.iloc[:,[0,3,6,9,12]]

In [195]:
df_clean.columns = ['zipcode','studio','one_bedroom','two_bedroom','three_bedroom']


In [196]:
df_clean.to_csv('rent_data.csv', index=False)

In [199]:
df_clean.dtypes()

TypeError: 'Series' object is not callable

In [201]:
df_clean[df_clean['zipcode'] == 27409]

,zipcode,studio,one_bedroom,two_bedroom,three_bedroom
9347,27409,1040,1080,1200,1530


In [200]:
df_clean.dtypes

zipcode          int64
studio           int64
one_bedroom      int64
two_bedroom      int64
three_bedroom    int64
dtype: object